In [33]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/home/img/Code/Python/AICourse/AILabs/lib/python3.13/site-packages/pyspark'

## Machine Learning project witout Mllib pipeline

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
        .appName("Titanic Data")\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 14:55:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df = (spark.read
.format("csv")
.option("header","true")
.load("data/titanic/train.csv"))

In [5]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [6]:
from pyspark.sql.functions import col, isnull, count, when

In [7]:
dataset = df.select(col("Survived").cast("float"),
col("Pclass").cast("float"),
col("Sex"),
col("Age").cast("float"),
col("Fare").cast("float"),
col("Embarked"))

In [8]:
dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [9]:
dataset = dataset.replace("?", None)\
    .dropna(how="any")

In [10]:
dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [11]:
dataset.show(3)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
+--------+------+------+----+-------+--------+
only showing top 3 rows



In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
dataset = StringIndexer(inputCol="Sex",
outputCol = "Gender",
handleInvalid="keep").fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol="Embarked",
outputCol = "Boarded",
handleInvalid="keep").fit(dataset).transform(dataset)

In [14]:
dataset.show(3)

+--------+------+------+----+-------+--------+------+-------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
|     1.0|   3.0|female|26.0|  7.925|       S|   1.0|    0.0|
+--------+------+------+----+-------+--------+------+-------+
only showing top 3 rows



In [15]:
dataset = dataset.drop("Sex")
dataset = dataset.drop("Embarked")

In [16]:
dataset.show(3)

+--------+------+----+-------+------+-------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|
+--------+------+----+-------+------+-------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|
+--------+------+----+-------+------+-------+
only showing top 3 rows



In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
feature_col = ["Pclass", "Age", "Fare", "Gender", "Boarded"]
assembler = VectorAssembler(inputCols=feature_col, outputCol="features")
transformed_data = assembler.transform(dataset)

In [19]:
transformed_data.show(5)

+--------+------+----+-------+------+-------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|            features|
+--------+------+----+-------+------+-------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+-------+--------------------+
only showing top 5 rows



In [20]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])
print(f"Numberof training samples {training_data.count()}")
print(f"Number of test samples {test_data.count()}")

Numberof training samples 564
Number of test samples 148


In [21]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features", maxDepth=5)
model = rf.fit(training_data)

In [22]:
predictions = model.transform(test_data)
predictions.show(5)

+--------+------+----+--------+------+-------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Age|    Fare|Gender|Boarded|            features|       rawPrediction|         probability|prediction|
+--------+------+----+--------+------+-------+--------------------+--------------------+--------------------+----------+
|     0.0|   1.0|19.0|   263.0|   0.0|    0.0|[1.0,19.0,263.0,0...|[12.1250832984182...|[0.60625416492091...|       0.0|
|     0.0|   1.0|28.0|    47.1|   0.0|    0.0|[1.0,28.0,47.0999...|[9.92616686181709...|[0.49630834309085...|       1.0|
|     0.0|   1.0|36.0|   78.85|   0.0|    0.0|[1.0,36.0,78.8499...|[12.6772593554947...|[0.63386296777473...|       0.0|
|     0.0|   1.0|38.0|153.4625|   0.0|    0.0|[1.0,38.0,153.462...|[13.3204113810683...|[0.66602056905341...|       0.0|
|     0.0|   1.0|45.0|    35.5|   0.0|    0.0|[1.0,45.0,35.5,0....|[13.8288256294744...|[0.69144128147372...|       0.0|
+--------+------+----+--------+-

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived",predictionCol="prediction", metricName="accuracy")

In [25]:
accuracy = evaluator.evaluate(predictions)
print(f"Model accuracy is : {accuracy}")

Model accuracy is : 0.8243243243243243


## Machine Learning with MLlib pipeline

In [26]:
spark = SparkSession\
    .builder\
        .appName("Titanic Data")\
            .getOrCreate()

In [27]:
spark

In [39]:
df = (spark.read
.format("csv")
.option("header","true")
.load("data/titanic/train.csv"))

In [40]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [41]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.ml.feature import OneHotEncoder

In [42]:
from pyspark.ml import Pipeline

In [48]:
dataset = df.select(col("Survived").cast("float"),
col("Pclass").cast("float"),
col("Sex"),
col("Age").cast("float"),
col("Fare").cast("float"),
col("Embarked"))
dataset = dataset.replace("?", None)\
    .dropna(how="any")
(train_df, test_df) = dataset.randomSplit([0.8,0.2], 11)
print(f"Number of train sample: {train_df.count()}")
print(f"Number of test samples: {test_df.count()}")

Number of train sample: 562
Number of test samples: 150


In [49]:
sex_indexer = StringIndexer(inputCol="Sex", outputCol="Gender")
embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Boarded")

In [50]:
inputColls = ["Pclass", "Age", "Fare", "Gender", "Boarded"]
outputCol = "features"
vector_assembler = VectorAssembler(inputCols=inputColls, outputCol=outputCol)

In [51]:
dt_model = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [52]:
pipeline = Pipeline(stages = [sex_indexer, embarked_indexer, vector_assembler, dt_model])
final_pipeline = pipeline.fit(train_df)

In [53]:
test_predictions = final_pipeline.transform(test_df)
test_predictions.show()

+--------+------+------+----+-------+--------+------+-------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-------+--------+------+-------+--------------------+--------------------+--------------------+----------+
|     0.0|   1.0|  male|19.0|  263.0|       S|   0.0|    0.0|[1.0,19.0,263.0,0...|[12.7970973502605...|[0.63985486751302...|       0.0|
|     0.0|   1.0|  male|21.0|77.2875|       S|   0.0|    0.0|[1.0,21.0,77.2874...|[11.1587307294312...|[0.55793653647156...|       0.0|
|     0.0|   1.0|  male|28.0|82.1708|       C|   0.0|    1.0|[1.0,28.0,82.1707...|[8.59697462952394...|[0.42984873147619...|       1.0|
|     0.0|   1.0|  male|29.0|   30.0|       S|   0.0|    0.0|[1.0,29.0,30.0,0....|[10.7034289599153...|[0.53517144799576...|       0.0|
|     0.0|   1.0|  male|29.0|   66.6|       S|  

In [54]:
accuracy = evaluator.evaluate(test_predictions)
print(f"Model accuracy is : {accuracy}")

Model accuracy is : 0.7866666666666666
